<a href="https://colab.research.google.com/github/soiqualang/colab/blob/master/Zalo_text_to_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/soiqualang/text-2-speech

In [13]:
import sys
import requests
import os
import random
import re
import time
import nltk
from urllib.parse import quote

In [14]:
class text2voice:

    def zalo_api(payload, voidid, speed):
        url = "https://zalo.ai/api/demo/v1/tts/synthesize"
        f = open("output.txt", "w")
        links = []
        for p in payload:
            text = quote(str(p))

            # text.encode('utf-8')  # Totally fine.
            payload = "input="+text+"&speaker_id=" + \
                voidid+"&speed="+speed+"&dict_id=0"
            headers = {
                "content-type": "application/x-www-form-urlencoded; charset=utf-8",
                "origin": "https://zalo.ai",
                "referer": "https://zalo.ai/experiments/text-to-audio-converter",

                "cookie": "zpsid=eMKnVbo-PZEvNHqtDTKIOgHQ7p4nrWzalI47O4wZJssuT3bRV_irVuyWFcWShorgrNnyH1sN7H_cHL08DySx4jayN3Kgv2SblZf95sovCHgQRaSg; zai_did=8k9uAj3FNiTevcSSryzXoYYo64d0o6V3AB4PHJ8q; zpsidleg=eMKnVbo-PZEvNHqtDTKIOgHQ7p4nrWzalI47O4wZJssuT3bRV_irVuyWFcWShorgrNnyH1sN7H_cHL08DySx4jayN3Kgv2SblZf95sovCHgQRaSg; zai_sid=lf2zTzCfGqIZbxznrofUGhhifo2eNnvBlxcP6va7P5c8xPue-bDyJDAnt0JxQqmvuOZmID4xQZJUyVnrp1Xs0xdtwLUAHM0ydQFdQl1IIGRigkzd; __zi=3000.SSZzejyD0jydXQcYsa00d3xBfxgP71AM8Tdbg8yB7SWftQxdY0aRp2gIh-QFHXF2BvMWxp0mDW.1; fpsend=149569; _zlang=vn"
            }

            response = requests.request(
                "POST", url, data=payload.encode('utf-8'), headers=headers)
            f.write(response.text+"\n")
            time.sleep(5)

        out = open('output.txt', 'r').read()

        links = re.findall(
            r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}.m3u8)', out)
        f.close()
        return links

    def split_text(payload):
        text = []
        long_sentence = []
        if len(payload) <= 200:
            text.append(payload)
            return text
        elif len(payload) > 200:
            sentences = nltk.sent_tokenize(payload)
            sub_para = ''

            for sen in sentences:
                if sub_para == '':
                    sub_para = sen

                elif sub_para != '':
                    if len(sub_para)+len(sen) <= 200 and sen != sentences[-1]:
                        sub_para = sub_para + " " + sen

                    elif len(sub_para)+len(sen) <= 200 and sen == sentences[-1]:
                        sub_para = sub_para + " " + sen
                        long_sentence.append(sub_para)

                    elif len(sub_para)+len(sen) > 200:
                        long_sentence.append(sub_para)
                        sub_para = ''
                        sub_para = sen

                    elif sen == sentences[-1]:
                        long_sentence.append(sub_para)

        return long_sentence

    def connect_audio(links):
        id = 1
        path = str(os.getcwd())
        full = path + '/tmp_audio/'
        command = 'cd '+full+' && rm -rf *'
        os.system(command)
        f = open('list_name.txt', 'w')
        for i in links:
            url = i
            des_fol = str(os.getcwd())+"/tmp_audio/"
            namefile = str(id)+".mp3"
            command = 'ffmpeg  -i '+url+' -ab 256k ' + des_fol + namefile + ' -y'
            id = id + 1
            os.system(command)
            f.write("file '"+full+namefile+"'\n")
        f.close()
        print("done")

    def get_links():
        out = open('output.txt', 'r').read()
        links = re.findall(
            r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}.m3u8)', out)
        return links

    def mer_audio(id):
        path_list = str(os.getcwd()) + "/list_name.txt"
        path = str(os.getcwd())+"/final_audio/"

        mp3_path = path+id+".mp3"
        command = 'ffmpeg -f concat -safe 0 -i ' + \
            path_list + ' -c copy '+mp3_path + ' -y'
        os.system(command)

        mp3_path = mp3_path.replace(os.getcwd(), '.')

        return mp3_path


class final_path_mp3():
    def get_path_mp3(id, payload, voiceid, speed):
        path = str(os.getcwd())+"/tmp_audio"
        if os.path.exists(path) == False:
            os.system("mkdir tmp_audio")
        path = str(os.getcwd()) + "/final_audio"
        if os.path.exists(path) == False:
            os.system("mkdir final_audio")
        data = text2voice.split_text(payload)
        text2voice.zalo_api(data, voiceid, speed)
        links = text2voice.get_links()
        text2voice.connect_audio(links)
        path = text2voice.mer_audio(id)

        return path

In [18]:
# if choice == "Nữ - Miền Nam":
#     voiceid = '1'
#     filename = "Nữ_Miền_Nam" + \
#         "_" + str(random.randint(1, 10000))
# elif choice == "Nam - Miền Nam":
#     voiceid = '2'
#     filename = "Nam_Miền_Nam" + \
#         "_" + str(random.randint(1, 10000))
# elif choice == "Nữ - Miền Bắc":
#     voiceid = '3'
#     filename = "Nữ_Miền_Bắc" + \
#         "_" + str(random.randint(1, 10000))
# elif choice == "Nam - Miền Bắc":
#     voiceid = '4'
#     filename = "Nam_Miền_Bắc" + \
#         "_" + str(random.randint(1, 10000))
# else:
#     filename = id
#     voiceid = '1'
# filename = str(filename)

voiceid = '1'
filename = "Nữ_Miền_Nam" + \
    "_" + str(random.randint(1, 10000))

payload = 'TP HCM - Đợt Covid-19 lần thứ tư khiến hàng loạt nghịch lý xuất hiện trên thị trường nhà ở và bị đẩy lên đỉnh điểm. 12 tháng qua, thị trường nhà ở TP HCM gặp nhiều thách thức về nguồn cung và thanh khoản do chịu tác động nặng nề bởi đợt Covid-19 lần thứ tư bùng phát. Trong bối cảnh kinh tế khó khăn đình trệ do tập trung phòng chống dịch, bất động sản cũng mất hai quý giữa năm gần như tê liệt hoàn toàn và chỉ gượng dậy chậm chạp vào quý cuối năm. Theo các chuyên gia, biến số Covid-19 đã khiến thị trường nhà ở năm 2021 giảm tốc, xuất hiện nhiều nghịch lý đáng quan ngại.'


final_path_mp3.get_path_mp3(
            id=filename, payload=payload, voiceid=voiceid, speed="1.0")

done


'./final_audio/Nữ_Miền_Nam_5859.mp3'

In [7]:
!pip install nltk

In [12]:
!python -m nltk.downloader punkt

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
